# Agents with LlamaIndex II - 

Sources [1](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/), [2](https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/agents/), [3](https://docs.llamaindex.ai/en/stable/examples/agent/custom_agent/), [4](https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent/), [5](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/agent_runner/), [6](https://medium.com/llamaindex-blog/data-agents-eed797d7972f), [LlamaHub Tools](https://llamahub.ai/?tab=tools)  
    
--- 
### Agents

An "agent" is an automated reasoning and decision engine. It takes in a user input/query and can make internal decisions for executing that query in order to return the correct result. The key agent components can include, but are not limited to:

+ Breaking down a complex question into smaller ones
+ Choosing an external Tool to use + coming up with parameters for calling the Tool
+ Planning out a set of tasks
+ Storing previously completed tasks in a memory module

LlamaIndex provides a comprehensive framework for building agents. This includes the following components:

+ Using agents with tools at a high-level to build agentic RAG and workflow automation use cases **(Previous Notebook)**
+ Low-level components for building and debugging agents
+ Core agent ingredients that can be used as standalone modules: query planning, tool use, and more **(Previous Notebook)**

### Tools

Having proper tool abstractions is at the core of building data agents. Defining a set of Tools is similar to defining any API interface, with the exception that these Tools are meant for agent rather than human use. We allow users to define both a Tool as well as a ToolSpec containing a series of functions under the hood.

When using an agent or LLM with function calling, the tool selected (and the arguments written for that tool) rely strongly on the tool name and description of the tools purpose and arguments. Spending time tuning these parameters can result in larges changes in how the LLM calls these tools.

A Tool implements a very generic interface - simply define __call__ and also return some basic metadata (name, description, function schema).
LlamaIndex offer a few different types of Tools:

+ FunctionTool: A function tool allows users to easily convert any user-defined function into a Tool. It can also auto-infer the function schema. **(Previous Notebook)**
+ QueryEngineTool: A tool that wraps an existing query engine. Note: since our agent abstractions inherit from BaseQueryEngine, these tools can also wrap other agents. **(Previous Notebook)**
+ Community contributed ToolSpecs that define one or more tools around a single service (like Gmail): see [LlamaHub](https://llamahub.ai/)
+ Utility tools for wrapping other tools to handle returning large amounts of data from a tool: see [OnDemandLoaderTool](https://docs.llamaindex.ai/en/stable/examples/tools/OnDemandLoaderTool/)

### Use Cases

The scope of possible use cases for agents is vast and ever-expanding. That said, here are some practical use cases that can deliver immediate value.

+ Agentic RAG: Build a context-augmented research assistant over your data that not only answers simple questions, but complex research tasks. (Previous Notebook)
+ SQL Agent: A subset of the above is a "text-to-SQL assistant" that can interact with a structured database. 
+ Workflow Assistant: Build an agent that can operate over common workflow tools like email, calendar.
+ Coding Assistant: Build an agent that can operate over code.

## Installing Packages

In [25]:
#%pip install -qU openai
#%pip install -qU llama-index
#%pip install -qU llama_hub
#%pip install -qU llama-index-experimental
#%pip install llama-index-agent-openai
#%pip install llama-index-readers-web
#%pip install llama-index-tools-google


#!pip install -q pypdf
#!pip install -q docx2txt

## Importing Packages

In [2]:
import os
#os.environ["OPENAI_API_KEY"] = "<the key>"

import sys
import shutil
import glob
import logging
from pathlib import Path
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import openai
import llama_index
from llama_index.llms.openai import OpenAI

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import SummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.experimental.query_engine import PandasQueryEngine

## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node

## LlamaIndex Templates
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.base.llms.types import ChatMessage, MessageRole

## LlamaIndex Agents
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

import nest_asyncio
nest_asyncio.apply()

In [3]:
import logging

#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

---

## 1- Function Tool  
A function tool is a simple wrapper around any existing function (both sync and async are supported!).  

In [4]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def subtract(x: int, y: int) -> int: 
    """Subtract the second number from the first number."""
    return (x - y)

def multiply(x: int, y: int) -> int: 
    """Multiply one number by the other number."""
    return (x * y)

def uppercase(x: str) -> str: 
    """Return the input string in uppercase."""
    return (x.upper())

add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
uppercase_tool = FunctionTool.from_defaults(fn=uppercase)

In [5]:
#from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

response = llm.predict_and_call(
    [add_tool, subtract_tool, multiply_tool, uppercase_tool], 
    "Tell me the output of 3 - 12 ", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: subtract with args: {"x": 3, "y": 12}
=== Function Output ===
-9
-9


---

## 2- [Coding Assistant and Interpreter](https://llamahub.ai/l/tools/llama-index-tools-code-interpreter?from=)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-code-interpreter/examples/code_interpreter.ipynb)

In [92]:
%pip install llama-index-tools-code_interpreter

In [113]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.code_interpreter import CodeInterpreterToolSpec

code_spec = CodeInterpreterToolSpec()
tools = code_spec.to_tool_list()
agent = OpenAIAgent.from_tools(tools, verbose=True)

In [114]:
print(agent.chat("Can you help me write some python code to pass to the code_interpreter tool"))

Added user message to memory: Can you help me write some python code to pass to the code_interpreter tool
Of course! What specific task or problem would you like the Python code to address?


In [115]:
print(agent.chat(
        """There is a california_housing_train.csv file in the `data` directory (relative path).
                 Can you write and execute code to tell me its columns?"""
    )
)

Added user message to memory: There is a california_housing_train.csv file in the `data` directory (relative path).
                 Can you write and execute code to tell me its columns?
=== Calling Function ===
Calling function: code_interpreter with args: {"code":"import pandas as pd\n\n# Load the dataset\ndata = pd.read_csv('data/california_housing_train.csv')\n\n# Get the columns of the dataset\ncolumns = data.columns\n\n# Print the columns\nprint(columns)"}
Got output: StdOut:
b"Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',\n       'total_bedrooms', 'population', 'households', 'median_income',\n       'median_house_value'],\n      dtype='object')\n"
StdErr:
b''

The columns in the `california_housing_train.csv` file are:
- longitude
- latitude
- housing_median_age
- total_rooms
- total_bedrooms
- population
- households
- median_income
- median_house_value

Let me know if you need any further assistance!


In [116]:
print(agent.chat("In which country are these latitudes and longitudes?"))

Added user message to memory: In which country are these latitudes and longitudes?
The latitudes and longitudes in the `california_housing_train.csv` file correspond to locations in the United States, specifically in the state of California. The dataset contains housing information for various locations within California.


In [117]:
print(agent.chat("Can you make a graph of the median income per total rooms?"))

Added user message to memory: Can you make a graph of the median income per total rooms?
=== Calling Function ===
Calling function: code_interpreter with args: {"code":"import matplotlib.pyplot as plt\n\n# Plotting the graph\nplt.figure(figsize=(10, 6))\nplt.scatter(data['total_rooms'], data['median_income'])\nplt.title('Median Income per Total Rooms')\nplt.xlabel('Total Rooms')\nplt.ylabel('Median Income')\nplt.grid(True)\nplt.show()"}
Got output: StdOut:
b''
StdErr:
b'Traceback (most recent call last):\n  File "<string>", line 5, in <module>\nNameError: name \'data\' is not defined\n'

=== Calling Function ===
Calling function: code_interpreter with args: {"code":"import pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load the dataset\ndata = pd.read_csv('data/california_housing_train.csv')\n\n# Plotting the graph\nplt.figure(figsize=(10, 6))\nplt.scatter(data['total_rooms'], data['median_income'])\nplt.title('Median Income per Total Rooms')\nplt.xlabel('Total Rooms')\nplt.ylabel(

In [118]:
print(agent.chat("I cant see the plot - can you save it locally with file name `output.png`?"))

Added user message to memory: I cant see the plot - can you save it locally with file name `output.png`?
=== Calling Function ===
Calling function: code_interpreter with args: {"code":"import pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load the dataset\ndata = pd.read_csv('data/california_housing_train.csv')\n\n# Plotting the graph\nplt.figure(figsize=(10, 6))\nplt.scatter(data['total_rooms'], data['median_income'])\nplt.title('Median Income per Total Rooms')\nplt.xlabel('Total Rooms')\nplt.ylabel('Median Income')\nplt.grid(True)\nplt.savefig('output.png')"}
Got output: StdOut:
b''
StdErr:
b''

The plot has been saved locally as `output.png`. You can download it using the following link: [Download output.png](sandbox:/output.png)

If you need any further assistance, feel free to ask!


---

## 3- [SQL Agent](https://llamahub.ai/l/tools/llama-index-tools-database?from=)  
[GitHub](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-database/examples/database.ipynb), [article](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo/)

In [130]:
%pip install llama-index-tools-database
%pip install sqlalchemy

In [164]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.database import DatabaseToolSpec
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    insert,
    text,
)

In [160]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [161]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

#### Define SQL Database   

We first define our SQLDatabase abstraction (a light wrapper around SQLAlchemy).

In [163]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

#### We add some testing data to our SQL database

In [165]:
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States",},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [166]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


#### Query Index  
We first show how we can execute a raw SQL query, which directly executes over the table.  

In [168]:
with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


#### Part 1: Text-to-SQL Query Engine¶

Once we have constructed our SQL database, we can use the NLSQLTableQueryEngine to construct natural language queries that are synthesized into SQL queries.

Note that we need to specify the tables we want to use with this query engine. If we don't the query engine will pull all the schema context, which could overflow the context window of the LLM.

This query engine should be used in any case where you can specify the tables you want to query over beforehand, or the total size of all the table schema plus the rest of the prompt fits your context window.

In [169]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [171]:
from IPython.display import Markdown
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo has the highest population with 13,960,000 residents.</b>

#### Part 2: Query-Time Retrieval of Tables for Text-to-SQL

If we don't know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we should store the table schema in an index so that during query time we can retrieve the right schema.

The way we can do this is using the SQLTableNodeMapping object, which takes in a SQLDatabase and produces a Node object for each SQLTableSchema object passed into the ObjectIndex constructor.  

In [172]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
#from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

#### Now we can take our SQLTableRetrieverQueryEngine and query it for our response.

In [173]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo has the highest population among all cities, with a population of 13,960,000.</b>

In [174]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[('Tokyo', 13960000)]

#### You could also add additional context information for each table schema you define.

In [ ]:
# manually set context text
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text))
]

#### Part 3: Text-to-SQL Retriever

So far our text-to-SQL capability is packaged in a query engine and consists of both retrieval and synthesis.

You can use the SQL retriever on its own. We show you some different parameters you can try, and also show how to plug it into our RetrieverQueryEngine to get roughly the same results.


In [175]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True
)

In [176]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [177]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** 474462d5-684a-4f29-b227-943666c629f5<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>

In [178]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

In [179]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [180]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** feb976f6-5517-4b73-aa99-700a915d4fe7<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** 972e5033-ae0c-40c6-af1f-2ccbda3c029d<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** 7ece5173-cfb9-4756-b185-b89e0d7b397e<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** b8d1b88a-068d-455f-aff5-c7a331d4ba61<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

#### Plug into our RetrieverQueryEngine

We compose our SQL Retriever with our standard RetrieverQueryEngine to synthesize a response. The result is roughly similar to our packaged Text-to-SQL query engines.

In [181]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [182]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [183]:
print(str(response))

The top 5 cities with the highest populations are:

1. Tokyo - 13,960,000
2. Seoul - 9,776,000
3. Toronto - 2,930,000
4. Chicago - 2,679,000


#### Using Database Tool

This tool connects to a database (using SQLAlchemy under the hood) and allows an Agent to query the database and get information about the tables.

In [188]:
db_tools = DatabaseToolSpec(sql_database)
agent = OpenAIAgent.from_tools(db_tools.to_tool_list())

agent.chat("What tables does this database contain").response

'This database contains the following table:\n- city_stats'

In [190]:
print(agent.chat("Describe the first table").response)

The table "city_stats" has the following schema:
- city_name: VARCHAR(16) NOT NULL
- population: INTEGER
- country: VARCHAR(16) NOT NULL
- PRIMARY KEY: city_name


In [191]:
print(agent.chat("Retrieve the first row of that table").response)

The first row of the "city_stats" table is:
- city_name: Toronto
- population: 2930000
- country: Canada


---

## 4- Community Tools

### A) [Wikipedia Tool](https://llamahub.ai/l/tools/llama-index-tools-wikipedia?from=tools)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-wikipedia/examples/wikipedia.ipynb)

In [ ]:
%pip install llama-index-tools-wikipedia

In [34]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.wikipedia import WikipediaToolSpec

tool_spec = WikipediaToolSpec()

agent = OpenAIAgent.from_tools(tool_spec.to_tool_list()) #, verbose=True,)

response= agent.chat("Who is Ben Afflecks spouse?")

#print(response.sources)
print(response.response)

Ben Affleck was married to actress Jennifer Garner from 2005 to 2018. They began dating in August 2004 and got married on June 29, 2005, in a private ceremony in Turks and Caicos. They have three children together: Violet Anne Affleck, born in December 2005, Fin Affleck, born in January 2009, and Samuel Garner Affleck, born in February 2012. They announced their separation in June 2015 and filed for divorce in April 2017, which was finalized in October 2018.


### B) [Arxiv Tool](https://llamahub.ai/l/tools/llama-index-tools-arxiv?from=tools)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-arxiv)  

In [ ]:
%pip install llama-index-tools-arxiv

In [32]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.arxiv.base import ArxivToolSpec

arxiv_tool = ArxivToolSpec()

agent = OpenAIAgent.from_tools(arxiv_tool.to_tool_list(), verbose=True,)

print(agent.chat("Whats going on with the superconductor lk-99"))

Added user message to memory: Whats going on with the superconductor lk-99
=== Calling Function ===
Calling function: arxiv_query with args: {"query":"superconductor LK-99"}
Got output: [Document(id_='74990fb1-9e32-4488-a618-3cd766e4782d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='http://arxiv.org/pdf/2307.16402v2: Synthesis of possible room temperature superconductor LK-99:Pb$_9$Cu(PO$_4$)$_6$O\nThe quest for room-temperature superconductors has been teasing scientists\nand physicists, since its inception in 1911 itself. Several assertions have\nalready been made about room temperature superconductivity but were never\nverified or reproduced across the labs. The cuprates were the earliest high\ntransition temperature superconductors, and it seems that copper has done the\nmagic once again. Last week, a Korean group synthesized a Lead Apatite-based\ncompound LK-99, showing a T$_c$ of above 400$^\\circ$K. The sig

### C) [DuckDuckGoSearch Tool](https://llamahub.ai/l/tools/llama-index-tools-duckduckgo?from=tools)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-duckduckgo/examples/duckduckgo_search.ipynb)  

In [39]:
%pip install llama-index-tools-duckduckgo

In [42]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

tool_spec = DuckDuckGoSearchToolSpec()

agent = OpenAIAgent.from_tools(DuckDuckGoSearchToolSpec().to_tool_list(), verbose=True,)
response = agent.chat("what are the latest developments in machine learning")

Added user message to memory: what are the latest developments in machine learning
=== Calling Function ===
Calling function: duckduckgo_full_search with args: {"query":"latest developments in machine learning","max_results":5}
Got output: [{'title': 'Machine learning | MIT News | Massachusetts Institute of Technology', 'href': 'https://news.mit.edu/topic/machine-learning', 'body': 'Machine learning. Download RSS feed: News Articles ... Such discoveries help researchers better understand the development of molecular complexity in space during star formation. ... To build a better AI helper, start by modeling the irrational behavior of humans. A new technique can be used to predict the actions of human or AI agents who ...'}, {'title': 'Latest Developments in Machine Learning: 2024 Update', 'href': 'https://waleed.dev/blog/latest-developments-in-machine-learning-2024-update', 'body': "Some of the latest developments in machine learning that are shaping the industry in 2024 include: Gene

In [44]:
print(response.response)

Here are some of the latest developments in machine learning:

1. [Machine learning | MIT News | Massachusetts Institute of Technology](https://news.mit.edu/topic/machine-learning): Discoveries in understanding the development of molecular complexity in space during star formation and modeling irrational behavior of humans for building better AI helpers.

2. [Latest Developments in Machine Learning: 2024 Update](https://waleed.dev/blog/latest-developments-in-machine-learning-2024-update): Generative AI models like DALL-E 3, Stable Diffusion, and Google's Imagen are producing highly realistic synthetic images and videos, impacting content creation, entertainment, art, and more.

3. [10 top AI and machine learning trends for 2024 | TechTarget](https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends): Trends in AI development and deployment strategies focusing on ethics, safety, and regulatory landscape, including Multimodal AI.

4. [What's next for AI in 20

In [45]:
agent.chat_history.clear()
print(agent.chat("Who is Goethe?"))

Added user message to memory: Who is Goethe?
=== Calling Function ===
Calling function: duckduckgo_instant_search with args: {"query":"Goethe"}
Got output: [{'icon': None, 'text': 'Johann Wolfgang von Goethe was a German polymath and writer, who is widely regarded as the greatest and most influential writer in the German language. His work has had a profound and wide-ranging influence on Western literary, political, and philosophical thought from the late 18th century to the present day. Goethe was a German poet, playwright, novelist, scientist, statesman, theatre director, and critic. His works include plays, poetry and aesthetic criticism, as well as treatises on botany, anatomy, and color. Goethe took up residence in Weimar in November 1775 following the success of his first novel, The Sorrows of Young Werther. He was ennobled by the Duke of Saxe-Weimar, Karl August, in 1782. Goethe was an early participant in the Sturm und Drang literary movement.', 'topic': None, 'url': 'https://e

### D) [Yahoo Finance Tool](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/tools/)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-yahoo-finance)  

In [48]:
%pip install llama-index-tools-yahoo-finance

In [52]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

tool_spec = YahooFinanceToolSpec()
agent = OpenAIAgent.from_tools(tool_spec.to_tool_list())

print(agent.chat("What is the price of Apple stock?").response)

The current price of Apple Inc. (AAPL) stock is $186.88.


In [51]:
print(agent.chat("What is the latest news about Apple?").response)

Here are some of the latest news titles about Apple Inc. (AAPL):
1. "As Nvidia Smashes Targets, This Mag 7 Stock Teases Buy Point, Record High"
2. "DOJ widens antitrust crackdown as it seeks a breakup of Live Nation"
3. "Magnificent Seven Stocks: Nvidia Stock Soars 10% On Earnings; Apple, Tesla Slide"
4. "Top Stock Reports for Apple, Mastercard & Amgen"
5. "Paris transit passes now available in iPhone’s Wallet app"
6. "Why Apple's $110 Billion Buyback Program Works"
7. "3 Blue Chip Stocks That You Can Buy and Hold for Years"
8. "Xiaomi Raises Target for Marquee EV in Signal of Confidence"


### E) [Open Weather Map Tool](https://llamahub.ai/l/tools/llama-index-tools-weather?from=tools)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-weather)  

In [65]:
%pip install llama-index-tools-weather
%pip install pyowm

In [155]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.weather import OpenWeatherMapToolSpec

os.environ["OPENWEATHER_API_KEY"]="your key"

## https://openweathermap.org/api
tool_spec = OpenWeatherMapToolSpec(key=os.environ["OPENWEATHER_API_KEY"])

agent = OpenAIAgent.from_tools(tool_spec.to_tool_list())

agent.chat("What is the temperature like in Paris?")
#agent.chat("What is the wind like in Budapest tomorrow?")

AgentChatResponse(response='I apologize for the inconvenience, but it appears that there is still an issue with accessing the weather information for Paris. The API key provided seems to be invalid.', sources=[ToolOutput(content='Error: Invalid API Key provided', tool_name='weather_at_location', raw_input={'kwargs': {'location': 'Paris, France'}}, raw_output=UnauthorizedError('Invalid API Key provided'), is_error=False), ToolOutput(content='Error: Invalid API Key provided', tool_name='weather_at_location', raw_input={'kwargs': {'location': 'Paris'}}, raw_output=UnauthorizedError('Invalid API Key provided'), is_error=False), ToolOutput(content='Error: Invalid API Key provided', tool_name='weather_at_location', raw_input={'kwargs': {'location': 'Paris, France'}}, raw_output=UnauthorizedError('Invalid API Key provided'), is_error=False), ToolOutput(content='Error: Invalid API Key provided', tool_name='weather_at_location', raw_input={'kwargs': {'location': 'Paris'}}, raw_output=Unauthoriz

## 5- [OnDemandLoaderTool](https://docs.llamaindex.ai/en/stable/examples/tools/OnDemandLoaderTool/)  

OnDemandLoaderTool is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.  

This tool takes in a BaseReader data loader, and when called will 1) load data, 2) index data, and 3) query the data.  

In this walkthrough, we show how to use the OnDemandLoaderTool to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [71]:
%pip install llama-index-readers-wikipedia

In [72]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

#### Define Tool¶

First it is defined the WikipediaReader. Note that the load_data interface to WikipediaReader takes in a list of pages.   
By default, this queries the Wikipedia search endpoint which will autosuggest the relevant pages.  
Then it wraps it into our OnDemandLoaderTool.  
By default since we don't specify the index_cls, a simple vector store index is initialized.  

In [73]:
reader = WikipediaReader()

tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

# run tool by itself
tool(["Berlin"], query_str="What's the arts and culture scene in Berlin?")

ToolOutput(content="Berlin's arts and culture scene is vibrant and diverse, with a rich history in performing arts, music, and festivals. The city boasts numerous theaters and stages, including iconic venues like the Deutsches Theater, Volksbühne, and Berliner Ensemble. Berlin is also home to three major opera houses - the Deutsche Oper, Berlin State Opera, and Komische Oper. The city hosts a variety of musical theater performances, contemporary dance shows, and symphony orchestras, with the Berlin Philharmonic Orchestra being one of the world's most renowned. Additionally, Berlin is known for its underground electronic music scene and hosts a wide range of festivals, including the Berlin International Film Festival, Karneval der Kulturen, and Berlin Pride, making it a hub for cultural events and celebrations.", tool_name='Wikipedia Tool', raw_input={'query': "What's the arts and culture scene in Berlin?"}, raw_output=Response(response="Berlin's arts and culture scene is vibrant and di

In [80]:
#from llama_index.indices.vector_store import VectorStoreIndex

reader = WikipediaReader()
tool = OnDemandLoaderTool.from_defaults(
    reader=reader,
    index_cls=VectorStoreIndex,
    index_kwargs={"dim": 768},
    name="Wikipedia",
    description="A tool for querying Wikipedia articles.",
)

In [89]:
response = tool(["City of Vienna"], query_str="What are the best universities in Vienna?")

In [90]:
response.content

'The best universities in Vienna include the University of Vienna, TU Wien, Vienna University of Economics and Business, and the Medical University of Vienna.'

## Also check [These Notebooks](https://github.com/run-llama/llama-hub/tree/main/llama_hub/tools/notebooks)